In [ ]:
from pandas import Series, DataFrame
import config
import mysql.connector
from datetime import datetime
from mysql.connector import errorcode

In [ ]:
#SQL credentials. Store in config file.

sql_host = config.sql_host
sql_user = config.sql_user
sql_password = config.sql_password

In [ ]:
# Returns tuple for each sql row.
def selectlast(granularity, time, rows):
    
    sql = ("SELECT * FROM (SELECT * FROM {} WHERE time <= '{}' "
           "ORDER BY time DESC LIMIT {})a ORDER BY time ASC;".format(granularity, time, rows))
    cnx = mysql.connector.connect(user=sql_user, password=sql_password,
                             host=sql_host, database='USDJPY')
    cursor = cnx.cursor()
    cursor.execute(sql)
    results = cursor.fetchall()
    cursor.close()
    cnx.close()
    return results;

In [ ]:
# Create dataframe from sql query result
def dataframe(sqltable):
    names = ['time', 'openAsk', 'closeAsk', 'lowAsk', 'highAsk', 'openBid', 'closeBid', 'lowBid', 'highBid', 'volume']
    df = DataFrame(sqltable, columns = names)
    df.set_index('time', inplace=True)
    return df

In [ ]:
# Adds simple moving average to dataframe
def add_sma(df, val):
    df['SMA{}'.format(val)] = df['openBid'].rolling(window=val).mean()

In [ ]:
df = dataframe(selectlast('H3', datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'), 15))
add_sma(df, 3)
add_sma(df, 10)
df

In [ ]:
# Use for analysis
def add_state(df):
    state = 'Above'
    for i in df.index:
        if df.ix[i, 'SMA3'] < df.ix[i, 'SMA10']:
            state = 'Below'
        else:
            state = 'Above'

        df.ix[i, 'State'] = state

In [ ]:
# Use for analysis
def rearrange_cols(df):
    return df[['time', 'openAsk', 'lowAsk', 'highAsk', 'closeAsk', 'openBid', 'lowBid', 'highBid',
             'closeBid', 'SMA3', 'SMA10', 'State',]]

In [ ]:
# change to work with json data

def crosses(df):
    crosses = 0

    for i in df.index[20:]:
        if check_trend(df, i):
            if df.ix[i-1,'State'] == 'Below' and df.ix[i, 'State'] == 'Above' : # True when SMA3 crosses over SMA10
                diff = ((df.ix[i, 'SMA10'] - df.ix[i-15, 'SMA10']) / df.ix[i, 'SMA10']) * 100
                spread = (df.ix[i, 'openAsk'] - df.ix[i, 'openBid']) *100
                print('{} at {} {}, Spread: {}'.format(
                        round(diff, 6), df.ix[i, 'time'],df.ix[i, 'date'], round(spread ,9)))
                crosses += 1

    print('Crossed above {} times.'.format(crosses))